# Lasso on the IBLI dataset

In this notebook, I will be using a Lasso Cross Validation model, and a Group Lasso model on the IBLI dataset WITHOUT THE WAVE VARIABLE

In [79]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from scipy import stats
import os
import seaborn as sns
from scipy.stats import f_oneway

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LassoCV, Lasso
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.base import clone

# Load and preprocess the data

In [80]:
#Load in the dataset
df = pd.read_excel("C:/Users/daanm/Documents/Universiteit Utrecht/Scriptie/cattle_df.xlsx")
df.head(15)

,Unnamed: 0,id,wave,afm_language,age_constant,agric_land,amh_language,educ_recoded_constant,eng_language,expend,...,main_info_source_recoded,religion_recoded,owns_phone,household_moved,purchase_bin,why_not_purchase_recoded,know_vip,trust_vip,advise_vip,ratio_insured_cattle
0,0,1,4,No,25,Yes,No,Never attended,No,0,...,Interpersonal Sources,Traditional/Wakefata,0,No,No,Lack of Awareness or Understanding,Yes,Yes,Yes,0.0
1,1,2,4,Yes,27,Yes,No,Adult Education,No,0,...,Interpersonal Sources,Traditional/Wakefata,0,No,No,"Financial, Practical, and Situational Constraints",Yes,Yes,Yes,0.0
2,2,3,4,Yes,29,No,No,Elementary,No,1,...,Professional and Organizational Sources,Christian,0,No,No,"Financial, Practical, and Situational Constraints",Yes,Yes,Yes,0.0
3,3,4,4,No,35,Yes,No,Never attended,No,0,...,Interpersonal Sources,Traditional/Wakefata,0,No,No,"Financial, Practical, and Situational Constraints",Yes,Yes,Yes,0.0
4,5,6,4,No,60,Yes,No,Never attended,No,0,...,Interpersonal Sources,Traditional/Wakefata,0,No,No,Lack of Awareness or Understanding,Yes,Yes,No,0.0
5,6,7,4,No,25,No,No,Never attended,No,0,...,Interpersonal Sources,Traditional/Wakefata,0,No,Yes,BOUGHT INSURANCE,Yes,Yes,Yes,0.0
6,7,8,4,No,61,Yes,No,Never attended,No,0,...,Interpersonal Sources,Christian,0,No,Yes,BOUGHT INSURANCE,Yes,Yes,Yes,0.0
7,8,9,4,No,45,Yes,No,Never attended,No,0,...,Interpersonal Sources,Traditional/Wakefata,0,No,No,Lack of Awareness or Understanding,Yes,Yes,No,0.0
8,9,10,4,No,30,Yes,No,Never attended,No,0,...,Professional and Organizational Sources,Traditional/Wakefata,0,No,No,"Financial, Practical, and Situational Constraints",Yes,Yes,Yes,0.0
9,11,12,4,No,30,No,No,Never attended,No,0,...,Interpersonal Sources,Christian,0,Yes,No,Lack of Awareness or Understanding,No,No,Yes,0.0


In [81]:
#Agreement has been made to drop the ratio column from this dataset since it introduces endogeneity

df = df.drop(['Unnamed: 0','advise_vip','purchase_bin','cs_cs_ratio_post_cattle','buy_nr_cattle','buy_cattle', 'ratio_insured_cattle', 'n_previd_cattle','wave', 'id'], axis=1)
columns_list = df.columns.tolist()
print(columns_list)

['afm_language', 'age_constant', 'agric_land', 'amh_language', 'educ_recoded_constant', 'eng_language', 'expend', 'irrigated_land_bin', 'cs_cs_diff_post_cattle', 'number_minors', 'educ_child_recoded', 'activity_child_recoded', 'household_description', 'number_adults', 'main_info_source_recoded', 'religion_recoded', 'owns_phone', 'household_moved', 'why_not_purchase_recoded', 'know_vip', 'trust_vip']


In [82]:
#Log+1 transforming the dataset to get a better distribution

df['cs_diff_log'] = np.log(np.abs(df['cs_cs_diff_post_cattle']) + 1)
df[['cs_cs_diff_post_cattle', 'cs_diff_log']].head()

,cs_cs_diff_post_cattle,cs_diff_log
0,-188.170624,5.242649
1,-850.367249,6.746844
2,-5219.686523,8.560384
3,-157.544266,5.066034
4,-158.881271,5.074431


# Deploying the Lasso Model
https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LassoCV.html

In [83]:
y = df['cs_diff_log']
X = df.drop(columns=['cs_diff_log', 'cs_cs_diff_post_cattle'])

In [84]:
y.head()

0    5.242649
1    6.746844
2    8.560384
3    5.066034
4    5.074431
Name: cs_diff_log, dtype: float64

In [58]:
categorical = X.select_dtypes(include=['object','category']).columns.tolist()
numerical = X.select_dtypes(include=['int64','float64']).columns.tolist()

In [59]:
#Preprocess the pipeline
preprocessor = ColumnTransformer([
    ('num', Pipeline([
        ('imputer', SimpleImputer(strategy='mean')),
        ('scaler', StandardScaler())
    ]), numerical),
    
#Encode the categorical columns    
    ('cat', Pipeline([
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('onehot', OneHotEncoder(handle_unknown='ignore'))
    ]), categorical)
])


In [60]:
#Create a Train Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [61]:
lasso_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('lasso', LassoCV(cv=5, random_state=42))
])

lasso_pipeline.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['age_constant', 'expend',
                                                   'irrigated_land_bin',
                                                   'number_minors',
                                                   'number_adults',
                                                   'owns_phone']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['afm_language', 'agric_land',
                                                   'amh_language',
                                                   'educ_recoded_constant',
                                                   'eng_language',
                                                   'educ_child_recoded',
                                                   'activity_child_recoded',
                                                   'household_description',
                                                   'main_info_source_recoded',
                                                   'religion_recoded',
                                                   'household_moved',
                                                   'why_not_purchase_recoded',
                                                   'know_vip',
                                                   'trust_vip'])])),
                ('lasso', LassoCV(cv=5, random_state=42))])

# Results

In [62]:
y_pred = lasso_pipeline.predict(X_test)

print("R²:", r2_score(y_test, y_pred))
print("MSE:", mean_squared_error(y_test, y_pred))

R²: -0.005335293499517224
MSE: 2.7348283928226134


In [63]:
lasso_model = lasso_pipeline.named_steps['lasso']
coefficients = lasso_model.coef_
print("Number of non-zero coefficients:", np.sum(coefficients != 0))

Number of non-zero coefficients: 18


In [64]:
feature_names = lasso_pipeline.named_steps['preprocessor'].get_feature_names_out()

coef = lasso_pipeline.named_steps['lasso'].coef_
coef_df = pd.DataFrame({
    'Feature': feature_names,
    'Coefficient': coef
})

#Remove the 0 coefficients
non_zero_coef_df = coef_df[coef_df['Coefficient'] != 0]

# Sort the coefficients
top_features = non_zero_coef_df.reindex(non_zero_coef_df.Coefficient.abs().sort_values(ascending=False).index)

print(top_features.head(20))


                                              Feature   Coefficient
28  cat__household_description_Fully settled: The ...  2.980695e-01
45                                  cat__trust_vip_No -2.198609e-01
31  cat__main_info_source_recoded_Interpersonal So...  1.237380e-01
42  cat__why_not_purchase_recoded_Lack of Awarenes... -1.196336e-01
43                                   cat__know_vip_No -6.627483e-02
36         cat__religion_recoded_Traditional/Wakefata -6.512461e-02
6                                cat__afm_language_No -6.191545e-02
8                                  cat__agric_land_No -5.998381e-02
5                                     num__owns_phone -4.825179e-02
2                             num__irrigated_land_bin  2.656945e-02
0                                   num__age_constant -1.895026e-02
25                cat__activity_child_recoded_Student  1.552986e-02
3                                  num__number_minors -2.012885e-03
19                 cat__educ_child_recoded_Eleme

# Grid search with Cross Validation

In [65]:
from sklearn.model_selection import GridSearchCV

#Define the new pipeline
lasso_pipeline_gs = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('lasso', Lasso(max_iter=10000))
])

#Define the grid
param_grid = {'lasso__alpha': [0.01,0.009,0.007,0.006,0.005]}

#Perform Grid Search

grid_search = GridSearchCV(lasso_pipeline_gs, param_grid, cv=5, scoring='r2')
grid_search.fit(X_train,y_train)

print("Best alpha: ", grid_search.best_params_)
print("Best R²: ", grid_search.best_score_)

best_model = grid_search.best_estimator_



Best alpha:  {'lasso__alpha': 0.01}
Best R²:  0.01138033838712329


In [66]:
lasso_mae_list = []
lasso_rmse_list = []
lasso_mse_list = []

for alpha in param_grid['lasso__alpha']:
    model = Pipeline(steps=[
        ('preprocessor', clone(preprocessor)),
        ('lasso', Lasso(alpha=alpha, max_iter=10000))
    ])
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    mae = mean_absolute_error(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    mse = mean_squared_error(y_test, y_pred)

    lasso_mae_list.append(mae)
    lasso_rmse_list.append(rmse)
    lasso_mse_list.append(mse)

In [67]:
y_pred = best_model.predict(X_test)
r2_test = r2_score(y_test,y_pred)
mae_test = mean_absolute_error(y_test, y_pred)

print(f"Test R²: {r2_test:.4f}")
print(f"Test MAE: {mae_test:.4f}")

Test R²: -0.0080
Test MAE: 1.2043


# Group Lasso
https://group-lasso.readthedocs.io/en/latest/index.html

For this Group Lasso, I will be grouping all of the dummy variables created by the encoding.

In [68]:
from group_lasso import GroupLasso

In [69]:
preprocessor_group = ColumnTransformer([
    ('num', StandardScaler(), numerical),
    ('cat', OneHotEncoder(drop='first', sparse_output=False), categorical)
])
X_processed_group = preprocessor.fit_transform(X)

#Create the groups
group_list = []
group_id = 0

#create a group for each numerical value
for _ in numerical:
    group_list.append(group_id)
    group_id += 1

ohe = preprocessor.named_transformers_['cat']
cat_feature_names = ohe.get_feature_names_out(categorical)

#Create a group for each categorical value and dummy
grouped_counts = {}
for feature in cat_feature_names:
    prefix = feature.split("_")[0]
    grouped_counts[prefix] = grouped_counts.get(prefix,0) + 1

for _, count in grouped_counts.items():
    group_list.extend([group_id] * count)
    group_id += 1

groups = np.array(group_list)
print(groups)

[ 0  1  2  3  4  5  6  6  7  7  8  8  9  9  9  9  9  9  9  9  9 10 10 11
 11 11 11 11 12 12 12 12 12 13 13 13 14 14 14 15 15 15 15 16 16 17 17]


In [70]:
X_train, X_test, y_train, y_test = train_test_split(X_processed_group, y, test_size=0.2, random_state=42)

In [71]:
group_lasso = GroupLasso(groups=groups, group_reg=0.005, l1_reg=0.05,
                         scale_reg="group_size", supress_warning=True, n_iter=1000)

group_lasso.fit(X_train, y_train)

y_pred = group_lasso.predict(X_test)

print("R^2:", r2_score(y_test, y_pred))
print("MSE:", mean_squared_error(y_test, y_pred))
print("Number of non-zero coefficients:", np.sum(group_lasso.coef_ != 0))

R^2: -0.0017233227624464664
MSE: 2.7250026956749482
Number of non-zero coefficients: 0


In [72]:
feature_names = df.drop('cs_diff_log', axis=1).columns

non_zero_indices = np.where(group_lasso.coef_ != 0)[0]

non_zero_features = [(feature_names[i], group_lasso.coef_[i]) for i in non_zero_indices]

non_zero_df = pd.DataFrame(non_zero_features, columns=["Feature", "Coefficient"])
non_zero_df = non_zero_df.sort_values(by="Coefficient", key=abs, ascending=False)

print(non_zero_df)

Empty DataFrame
Columns: [Feature, Coefficient]
Index: []


# Grid search on Group Lasso

In [73]:
# Define the parameter grid

#Decides the penalty on groups
group_reg_values = [0.01, 0.05, 0.1,0.001 ,0.07] 

#Lasso penalty, decides penalty on individual features
l1_reg_values = [0.001, 0.005, 0.01, 0.05, 0.1]


best_score = -np.inf
best_params = None
results = []

In [74]:
X_train, X_val, y_train, y_val = train_test_split(X_processed_group, y, test_size=0.2, random_state=42)

In [75]:
# Manual grid search
for group_reg in group_reg_values:
    for l1_reg in l1_reg_values:
        model = GroupLasso(
            groups=groups,
            group_reg=group_reg,
            l1_reg=l1_reg,
            scale_reg="group_size",
            supress_warning=True,
            n_iter=1000
        )
        model.fit(X_train, y_train)
        y_pred = model.predict(X_val)
        score = r2_score(y_val, y_pred)
        mse = mean_squared_error(y_val, y_pred)
        rmse = np.sqrt(mse)
        mae = mean_absolute_error(y_val, y_pred)

        results.append({
            'group_reg': group_reg,
            'l1_reg': l1_reg,
            'r2_score': score,
            'mse': mse,
            'rmse': rmse,
            'mae': mae
        })

        if score > best_score:
            best_score = score
            best_params = (group_reg, l1_reg)

In [76]:
print(f" Best R² Score: {best_score:.4f} with group_reg={best_params[0]}, l1_reg={best_params[1]}")

results_df = pd.DataFrame(results)
print(results_df.sort_values(by="r2_score", ascending=False))

 Best R² Score: -0.0011 with group_reg=0.07, l1_reg=0.05
    group_reg  l1_reg  r2_score       mse      rmse       mae
23      0.070   0.050 -0.001089  2.723276  1.650235  1.198102
20      0.070   0.001 -0.001091  2.723282  1.650237  1.198116
17      0.001   0.010 -0.001204  2.723591  1.650331  1.198306
8       0.050   0.050 -0.001520  2.724449  1.650591  1.200439
4       0.010   0.100 -0.001522  2.724456  1.650593  1.200453
13      0.100   0.050 -0.001581  2.724615  1.650641  1.200747
18      0.001   0.050 -0.001581  2.724615  1.650641  1.200747
11      0.100   0.005 -0.001582  2.724618  1.650642  1.200751
3       0.010   0.050 -0.001722  2.724998  1.650757  1.201432
9       0.050   0.100 -0.001722  2.724999  1.650757  1.201433
10      0.100   0.001 -0.001722  2.724999  1.650757  1.201433
6       0.050   0.005 -0.001722  2.724999  1.650757  1.201435
5       0.050   0.001 -0.001722  2.725000  1.650757  1.201436
7       0.050   0.010 -0.001723  2.725001  1.650758  1.201437
21      0.070

Group Lasso does not seem to work as good? What groups did Sami use?

# Visualisations for comparison

In [77]:
lasso_results_df = pd.DataFrame(grid_search.cv_results_)
lasso_results_df = lasso_results_df[['param_lasso__alpha', 'mean_test_score']]
lasso_results_df.columns = ['l1_reg', 'r2_score']
lasso_results_df['mae'] = lasso_mae_list
lasso_results_df['mse'] = lasso_mse_list
lasso_results_df['rmse'] = lasso_rmse_list

group_lasso_results_df = pd.DataFrame(results)

lasso_results_df['model'] = 'Lasso'
group_lasso_results_df['model'] = 'Group Lasso'

combined_results = pd.concat([lasso_results_df, group_lasso_results_df], ignore_index=True)

In [78]:
combined_results_sorted = combined_results.sort_values(by='r2_score', ascending=False)

styled_table = combined_results_sorted.style\
    .background_gradient(subset=['r2_score'], cmap='Greens')\
    .format({'l1_reg': '{:.2e}', 'r2_score': '{:.4f}'})\
    .set_caption("Comparing Lasso and Group Lasso R² Scores")

styled_table


,l1_reg,r2_score,mae,mse,rmse,model,group_reg
0,1.00e-02,0.0114,1.204273,2.741947,1.655882,Lasso,nan
1,9.00e-03,0.0111,1.205516,2.744963,1.656793,Lasso,nan
2,7.00e-03,0.0104,1.208426,2.752222,1.658982,Lasso,nan
3,6.00e-03,0.0099,1.210366,2.757804,1.660664,Lasso,nan
4,5.00e-03,0.0094,1.212460,2.764194,1.662586,Lasso,nan
28,5.00e-02,-0.0011,1.198102,2.723276,1.650235,Group Lasso,0.070000
25,1.00e-03,-0.0011,1.198116,2.723282,1.650237,Group Lasso,0.070000
22,1.00e-02,-0.0012,1.198306,2.723591,1.650331,Group Lasso,0.001000
13,5.00e-02,-0.0015,1.200439,2.724449,1.650591,Group Lasso,0.050000
9,1.00e-01,-0.0015,1.200453,2.724456,1.650593,Group Lasso,0.010000
